In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoMouseEnhancers
from genomic_benchmarks.models.torch_cnn import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config 


## Config

In [3]:
config = {
    "use_padding": True,
    "run_on_gpu": True,
    "dataset": DemoMouseEnhancers,
    "number_of_classes": 2,
    "dataset_version": 0,
    "force_download": False,
    "epochs": 15,
    "embedding_dim": 100,
    "batch_size": 32,
#   vocabulary that is not present in the training set but is present in the test set
    "vocab_to_add": ["N"],
}
check_config(config)

config is correct


## Choose the dataset

In [4]:
get_dataset_fn = config["dataset"]
train_dset = get_dataset_fn('train', force_download=config["force_download"], version=config["dataset_version"])

Reference /home/jupyter/.genomic_benchmarks/fasta/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/21 [00:00<?, ?it/s]

## Tokenizer and vocab

In [5]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 8
{'N': 5, '<pad>': 7, 'T': 3, 'C': 4, 'A': 2, '<eos>': 6, 'G': 1, '<bos>': 0}


## Dataloader and batch preparation

In [6]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  4707
not all sequences are of the same length


## Model

In [7]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Training

In [8]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/home/jupyter/genomic_benchmarks/notebooks/utils.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 72.1%, Avg loss: 0.611164 

Epoch 1
Train metrics: 
 Accuracy: 72.7%, Avg loss: 0.609621 

Epoch 2
Train metrics: 
 Accuracy: 76.5%, Avg loss: 0.600974 

Epoch 3
Train metrics: 
 Accuracy: 74.1%, Avg loss: 0.603195 

Epoch 4
Train metrics: 
 Accuracy: 75.9%, Avg loss: 0.596156 

Epoch 5
Train metrics: 
 Accuracy: 78.1%, Avg loss: 0.592202 

Epoch 6
Train metrics: 
 Accuracy: 79.1%, Avg loss: 0.605551 

Epoch 7
Train metrics: 
 Accuracy: 80.3%, Avg loss: 0.585583 

Epoch 8
Train metrics: 
 Accuracy: 79.1%, Avg loss: 0.589198 

Epoch 9
Train metrics: 
 Accuracy: 81.3%, Avg loss: 0.586158 

Epoch 10
Train metrics: 
 Accuracy: 80.2%, Avg loss: 0.589761 

Epoch 11
Train metrics: 
 Accuracy: 81.6%, Avg loss: 0.582406 

Epoch 12
Train metrics: 
 Accuracy: 80.4%, Avg loss: 0.581917 

Epoch 13
Train metrics: 
 Accuracy: 80.8%, Avg loss: 0.587206 

Epoch 14
Train metrics: 
 Accuracy: 74.7%, Avg loss: 0.609546 



## Testing

In [9]:
test_dset = get_dataset_fn('test', force_download=config["force_download"], version=config["dataset_version"])
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Reference /home/jupyter/.genomic_benchmarks/fasta/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/21 [00:00<?, ?it/s]

test_loss  4.9486236572265625
num_batches 8
correct 180
size 242
Test Error: 
 Accuracy: 74.4%, Avg loss: 0.618578 

